# 定义一个网络(LeNet)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

只需要定义 forward 方法，反向传播方法会自动定义

In [3]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        # 输入深度为1，输出深度为6，卷积核大小为5
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self, x):
        # 最大池化，如果为方形，可以仅仅指定一个边
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1,self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


可学习的参数会通过net.parameters()方法返回

In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


定义一个输入

In [9]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[-0.0610, -0.0137,  0.0572,  0.0666,  0.0813, -0.0077,  0.0296, -0.0047,
         -0.0773, -0.0253]], grad_fn=<AddmmBackward>)


清零所有的梯度缓存，然后随机梯度反向传播

In [10]:
net.zero_grad()
out.backward(torch.randn(1, 10))

## 注意
torch.nn只支持小批量处理（mini-batches）。整个torch.nn包只支持小批量样本的输入，不支持单个样本。

比如，nn.Conv2d 接受一个4维的张量，即nSamples x nChannels x Height x Width

如果是一个单独的样本，只需要使用input.unsqueeze(0)来添加一个“假的”批大小维度。

# 损失函数

In [12]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()
loss = criterion(output,target)
print(loss)

tensor(1.6076, grad_fn=<MseLossBackward>)


In [13]:
print(loss.grad_fn)

# 反向传播

In [14]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0200, -0.0013,  0.0205, -0.0095,  0.0128, -0.0104])


# 更新网络权重
每次都要清零，不然会累加

In [17]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()